In [17]:
import numpy as np
import pandas as pd
import skyfield
from skyfield.api import load
from skyfield.positionlib import ICRF, Barycentric
from astropy.units import deg, au, km, meter, day, minute, second
from astropy.coordinates import GCRS, BarycentricMeanEcliptic, HeliocentricMeanEcliptic
import matplotlib.pyplot as plt
import astro_utils
from astro_utils import jd_to_mjd, radec2dir, radec_diff, qv2radec, qvrel2radec

### Observation of Earth and Mars according to JPL

In [2]:
# Load the JPL RA and DEC for Mars as fixed width file
df = pd.read_fwf('../data/jpl/testing/observe-mars-earth-geocenter.txt')

# Display the dataframe
df

,Date,Time,JulianDate,RA,DEC,RA_apparent,DEC_apparent delta,delta_dot,light_time
0,2010-Jan-01,00:00,2455197.5,142.327061,18.799029,142.475968,18.752303941 0.73883173669463,-8.970408,6.144676
1,2010-Jan-02,00:00,2455198.5,142.179197,18.888379,142.328371,18.841710905 0.73372184383589,-8.724067,6.102178
2,2010-Jan-03,00:00,2455199.5,142.017468,18.981461,142.166897,18.934867206 0.72875633894059,-8.470333,6.060882
3,2010-Jan-04,00:00,2455200.5,141.841890,19.078173,141.991560,19.031670832 0.72393956594361,-8.208951,6.020822
4,2010-Jan-05,00:00,2455201.5,141.652504,19.178403,141.802402,19.132004243 0.71927600922442,-7.939695,5.982036
...,...,...,...,...,...,...,...,...,...
3648,2019-Dec-28,00:00,2458845.5,233.190100,-18.727810,233.467326,-18.791714023 2.21286843425630,-12.191696,18.403866
3649,2019-Dec-29,00:00,2458846.5,233.876730,-18.895847,234.154585,-18.958738129 2.20580870839279,-12.255895,18.345152
3650,2019-Dec-30,00:00,2458847.5,234.565048,-19.061459,234.843518,-19.123323188 2.19871232427735,-12.318649,18.286133
3651,2019-Dec-31,00:00,2458848.5,235.255049,-19.224614,235.534120,-19.285436340 2.19158011449378,-12.379967,18.226816


In [3]:
# Extract obstime_jd, ra, and dec from DataFrame
obstime_jd = df.JulianDate.values
ra_jpl = df.RA.values
dec_jpl = df.DEC.values

# Vector of observation times in MJD format
obstime_mjd = jd_to_mjd(obstime_jd)

### Position & Observation of Earth and Mars according to Skyfield

In [4]:
# Manually load planetary positions using de435
planets_sf = load('../data/jpl/ephemeris/de435.bsp')
earth_sf = planets_sf['earth']
mars_sf = planets_sf['mars barycenter']

# load timescale
ts = load.timescale()

# Generate vector of observation times in Skyfield format
obstime_sf = ts.tt_jd(obstime_jd)

In [12]:
# Observe mars from earth with Skyfield
obs_sf = earth_sf.at(obstime_sf).observe(mars_sf)

# Build Skyfield angle arrays (RA, DEC) and distance array (delta)
ra_sf_aa, dec_sf_aa, delta_sf_da = obs_sf.radec()

# Extract degrees and AU to get plain arrays
ra_sf = ra_sf_aa._degrees
dec_sf = dec_sf_aa._degrees
delta_sf = delta_sf_da.au

### Position of Earth & Mars According to JPL: Same as Skyfield (Using JPL Ephemeris)

In [13]:
# Load planetary positions and velocities by querying the Skyfield JPL ephemeris interface
# Create them as arrays with bundled astropy units of au and km / second

# Earth
q_earth = earth_sf.at(obstime_sf).position.au * au
v_earth = earth_sf.at(obstime_sf).velocity.km_per_s * km / second

# Mars
q_mars = mars_sf.at(obstime_sf).position.au * au
v_mars = mars_sf.at(obstime_sf).velocity.km_per_s * km / second

### Compare Skyfield vs JPL

In [14]:
print(f'Mean Angle Difference: JPL vs. Skyfield')
diff_sf = radec_diff('JPL', 'Skyfield', ra1=ra_jpl, dec1=dec_jpl, ra2=ra_sf, dec2=dec_sf, 
                     obstime_mjd=obstime_mjd, verbose=False)
diff_mean = np.mean(diff_sf)
print(f'{diff_mean:5.3f} seconds')

Mean Angle Difference: JPL vs. Skyfield
1.598 seconds


In [8]:
# Day with biggest error
idx = np.argmax(diff_sf)
print(f'Date with largest error:')
print(f'index    = {idx}')
print(f'MJD      = {obstime_mjd[idx]}')
print(f'Date     = {astro_utils.mjd_to_date(obstime_mjd[idx])}')
print()
print(f'ra_sf    = {ra_sf[idx]:10.6f}')
print(f'ra_jpl   = {ra_jpl[idx]:10.6f}')
print(f'ra diff  = {ra_sf[idx]-ra_jpl[idx]:10.6f}')
print()
print(f'dec_sf   = {dec_sf[idx]:10.6f}')
print(f'dec_jpl  = {dec_jpl[idx]:10.6f}')
print(f'dec diff = {dec_sf[idx]-dec_jpl[idx]:10.6f}')
print()
print(f'AngleDiff= {diff_sf[idx]:7.3f}    seconds')

Date with largest error:
index    = 1121
MJD      = 56318.0
Date     = 2013-01-26

ra_sf    = 326.894343
ra_jpl   = 326.894940
ra diff  =  -0.000597

dec_sf   = -14.453341
dec_jpl  = -14.453133
dec diff =  -0.000208

AngleDiff=   2.213    seconds


### Observation of Mars from Earth Using MSE Functions Built on astropy Transforms

In [20]:
ra_mse1, dec_mse1, r_mse1 = qv2radec(q_mars, v_mars, obstime_mjd, BarycentricMeanEcliptic)

In [21]:
print(f'Mean Angle Difference: JPL vs. MSE qv2radec')
diff_sf = radec_diff('JPL', 'MSE', ra1=ra_jpl, dec1=dec_jpl, ra2=ra_mse1, dec2=dec_mse1, 
                     obstime_mjd=obstime_mjd, verbose=False)
diff_mean = np.mean(diff_sf)
print(f'{diff_mean:5.3f} seconds')

Mean Angle Difference: JPL vs. MSE qv2radec
55183.354 seconds
